In [2]:
import random
import time
import pandas as pd
import requests
from scrapy.http import TextResponse
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 크롤링한 맛집 데이터 로드
# df = pd.read_excel("./naver_df.xlsx")
# df.head()

,id,dong,name,tel,addr,bizhour,category,context,menu,reviewcount,microreview,thumbnail,lng,lat,rating
0,37450428,사직동,박순례 손말이고기 산정집 광화문점,02-310-9636,서울특별시 종로구 당주동 3-2 선덕빌딩 2층,일요일 휴무 명절연휴 휴무 | 매일 11:30~14:00 점심 | 매일 14:00~...,"['한식', '소고기구이']","['시래기', '고기말이', '한우', '고급스러운', '나들이']","한우말이고기 21,000 | 한우내장볶음 21,000 | 한우내장전골 25,000 ...",203,[],https://ldb-phinf.pstatic.net/20200930_73/1601...,126.974200,37.572380,NaN
1,1934318307,사직동,한옥 달,010-8706-3392,서울특별시 종로구 체부동 68,평일 11:30~22:00 (L.O. 20:30) | 평일 15:30~17:00 B...,"['이탈리아음식', '스파게티,파스타전문']",[],"단호박 크림 스프 15,000 | 아보카도 허머스 16,000 | 리코타 치즈 샐러...",322,[],https://ldb-phinf.pstatic.net/20190808_53/1565...,126.971983,37.576946,NaN
2,19489992,사직동,할매집,02-735-2608,서울특별시 종로구 내자동 108,매일 12:00~21:30,"['한식', '족발,보쌈']","['감자탕', '족발', '미슐랭가이드', '우아한', '콩국수']","족발 30,000 | 감자탕(중) 25,000 | 감자탕(대) 30,000",390,"['콩나물 넣은 감자탕', '구수하고 푸짐한 감자탕을 먹을 수 있는 식신로드 맛집'...",http://blogfiles.naver.net/20150910_18/xxxyeji...,126.971475,37.575660,NaN
3,1010940645,사직동,디퍼카페테리아,010-9160-4482,서울특별시 종로구 체부동 2-2,주말 12:00~19:30 | 수요일 12:00~19:30 | 목요일 12:00~1...,"['음식점', '카페,디저트']",[],"아메리카노 3,500 | 카페라떼 4,000 | 그라니따 5,500 | 스폴리아텔라...",39,[],https://ldb-phinf.pstatic.net/20200908_251/159...,126.970393,37.579038,NaN
4,676217924,사직동,후라토식당 경복궁 본점,02-730-7090,서울특별시 종로구 당주동 5,매일 11:30~22:00 평일 브레이크타임 15:00-17:00,"['일식', '일식당']","['오므라이스', '데이트', '일본가정식', '프라이빗', '돈가스']","규카츠 15,000 | 반숙 오므라이스 11,000 | 야마가타 민치카레 12,00...",497,[],https://ldb-phinf.pstatic.net/20200804_260/159...,126.974311,37.572032,NaN


In [4]:
# 네이버 지도에서 크롤링한 id값으로 네이버 플레이스에서 평점 크롤링
params = {'cookie' : 'NNB=LUURYNMJ4C6F6; nx_ssl=2; NRTK=ag#30s_gr#4_ma#2_si#2_en#2_sp#2; theme-promotion-november-notice-closed-count=1; page_uid=UJ0nKsp0YidssjcF2NRssssssf0-196483; BMR=s=1606280225697&r=https%3A%2F%2Fm.map.naver.com%2Fsearch2%2Fsearch.nhn%3Fquery%3D%25EC%2582%25AC%25EC%25A7%2581%25EB%258F%2599%2520%25EB%25A7%259B%25EC%25A7%2591%26sm%3Dhty%26style%3Dv5&r2=https%3A%2F%2Fm.map.naver.com%2Fsearch2%2Fsearch.nhn%3Fquery%3D%25EC%2582%25AC%25EC%25A7%2581%25EB%258F%2599%2520%25EB%25A7%259B%25EC%25A7%2591%26sm%3Dhty%26style%3Dv5; _naver_usersession_=SG1QrzcT5q6Ebt2OsL2srOJH; wcs_bt=sp_96fc91c9207ec0:1606394543|sp_9690f7f72d5c08:1606279853|80c9a70e2db3:1606225711',
         'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}
for i in range(0, len(df)+11, 11):
    for j in range(i, i+11):
        url = "https://m.place.naver.com/restaurant/{}/home".format(df["id"][j])
        try:
            response = requests.get(url, params=params)
            print(j, df["name"][j], response)
            dom = TextResponse(response.url, body=response.text, encoding='utf-8')
            df["rating"][j] = dom.xpath('//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]/div/span[1]/em/text()')[0].extract()
        except:
            print(j, df["name"][j], response)
            df["rating"][j] = False
    time.sleep(5)

0 박순례 손말이고기 산정집 광화문점 <Response [200]>
1 한옥 달 <Response [200]>
2 할매집 <Response [200]>
3 디퍼카페테리아 <Response [200]>
4 후라토식당 경복궁 본점 <Response [200]>
5 도토리브라더스 <Response [200]>
6 대접 광화문본점 <Response [200]>
7 심퍼티쿠시 경복궁점 <Response [200]>
8 밍고네식탁 <Response [200]>
9 가스트로통 <Response [200]>
10 오가와 <Response [200]>
11 몽쉐프 <Response [200]>
12 서촌계단집 <Response [200]>
13 암소서울 <Response [200]>
14 알레그리아 덕수궁 디팰리스점 <Response [200]>
15 티엔미미 <Response [200]>
16 체부동잔치집 <Response [200]>
17 카페아르크 <Response [200]>
18 올리바가든 <Response [200]>
19 사발 <Response [200]>
20 고가빈커리하우스 <Response [200]>
21 네기 실비 <Response [200]>
21 네기 실비 <Response [200]>
22 일품당 <Response [200]>
23 포비브라이트 광화문점 <Response [200]>
24 루뽀 <Response [200]>
25 힐사이드테이블 <Response [200]>
26 토속촌 <Response [200]>
27 포시즌스호텔서울 유유안 <Response [200]>
28 감성타코 광화문점 <Response [500]>
28 감성타코 광화문점 <Response [500]>
29 효자동초밥 <Response [200]>
30 코블러 <Response [200]>
31 애즈라이크 <Response [200]>
32 포시즌스 호텔 서울 보칼리노 <Response [200]>
33 칸다소바 <Response [200]>
34 텅댄스 <Response 

292 스타벅스 강북삼성병원점 <Response [200]>
293 독립문바지락칼국수 <Response [200]>
294 경복궁아구찜 <Response [200]>
295 브레댄코 강북삼성병원점 <Response [200]>
296 둘둘치킨 서대문점 <Response [200]>
297 하루한줄 <Response [200]>
298 본죽&비빔밥cafe 강북삼성병원점 <Response [200]>
299 우주라이크커피 서대문역점 <Response [200]>
300 묘미 <Response [200]>
301 다운타우너 안국 <Response [200]>
302 이잌 <Response [200]>
303 카페 노티드 안국 <Response [200]>
304 프릳츠 원서점 <Response [200]>
305 깡통만두 <Response [200]>
306 단상 <Response [200]>
307 어니언 안국 <Response [200]>
308 한식공간 <Response [200]>
309 소금집델리 안국 <Response [200]>
310 법원 <Response [200]>
311 우드앤브릭 <Response [200]>
312 도마 유즈라멘 <Response [200]>
313 레이어드 <Response [200]>
314 회화나무 <Response [200]>
315 하프커피 안국점 <Response [200]>
316 르꼬숑 <Response [200]>
317 리틀버틀러 <Response [200]>
318 떼레노 <Response [200]>
319 갤러리더스퀘어 <Response [200]>
320 콩지POT지 <Response [200]>
321 화양연화 <Response [200]>
322 반타이 <Response [200]>
323 땡스오트 안국 <Response [200]>
324 mishmash <Response [200]>
325 실리그린 <Response [200]>
326 만수옥 <Response [200]>
327 도시라쿤 <Re

583 한양국수돈삼오 <Response [200]>
584 해천횟집 <Response [200]>
585 명륜진사갈비 서울동묘점 <Response [200]>
586 홍두깨손칼국수 <Response [200]>
587 수원성갈비 <Response [200]>
588 동대문 본가 가마솥 설렁탕 <Response [200]>
589 목포낙지마을 <Response [200]>
590 재민이네 <Response [200]>
591 용두쌍둥이네쭈꾸미 <Response [200]>
592 광주식당 <Response [200]>
593 종가대박집 동대문역점 <Response [200]>
594 무한리필 고기굼터 <Response [200]>
595 호텔고기뷔페 <Response [200]>
596 동대문양육관점 <Response [200]>
597 대상하이샤브꼬치성 <Response [200]>
598 경양카츠 동대문점 <Response [200]>
599 손가네민물장어 <Response [200]>
600 동묘문 <Response [200]>
601 에베레스트 레스토랑 <Response [200]>
602 동묘집 <Response [200]>
603 정그리다 <Response [200]>
604 테르트르 <Response [200]>
605 차차 티클럽 <Response [200]>
606 창신동매운족발 <Response [200]>
607 동묘가라지 <Response [200]>
608 동묘마케트 <Response [200]>
609 동대문정육식당 <Response [200]>
610 동북화과왕 <Response [200]>
611 동래복국 <Response [200]>
612 동묘발전소 <Response [200]>
613 고기튀김 <Response [200]>
614 일품손짜장 <Response [200]>
615 동해해물탕 동대문직영점 <Response [200]>
616 낙산냉면 <Response [200]>
617 어탕국수 동묘점 <Response [200]>

875 독립맥주공장 <Response [200]>
876 한가람 본점 <Response [200]>
877 더 플라자 도원 <Response [200]>
878 천향원 <Response [200]>
879 속초생태집 <Response [200]>
880 서울웨스틴조선호텔 나인스게이트 <Response [200]>
881 롯데호텔서울 페닌슐라 라운지&바 <Response [200]>
882 광화문해물 <Response [200]>
883 델리카한스 <Response [200]>
884 디어와일드 <Response [200]>
885 르풀 <Response [200]>
886 마르셀 <Response [200]>
887 유림면 <Response [200]>
888 라그린 <Response [200]>
889 참치공방 시청점 <Response [200]>
890 복성각 덕수궁점 <Response [200]>
891 안즈 롯데백화점본점 <Response [200]>
892 더제이케이키친박스 시청역점 <Response [200]>
893 십원집 <Response [200]>
894 송원스키야키&샤브샤브 <Response [200]>
895 풍년 닭도리탕 <Response [200]>
896 덕수정 <Response [200]>
897 청송옥 <Response [200]>
898 진주회관 <Response [200]>
899 남도한식정든님고운님 <Response [200]>
900 중앙갈치식당 <Response [200]>
901 팔레드신 <Response [200]>
902 금산제면소 <Response [200]>
903 하이드앤시크 <Response [200]>
904 부원면옥 <Response [200]>
905 가메골손왕만두남대문본점 <Response [200]>
906 꽁시면관 <Response [200]>
907 라망시크레 <Response [200]>
908 육통령 명동본점 <Response [200]>
909 막내회집 본점 <Response [200]>
9

1155 전주집 <Response [200]>
1156 을지 장만옥 <Response [200]>
1157 은주정 <Response [200]>
1158 행2PM8PM <Response [200]>
1159 촙촙 <Response [200]>
1160 취향로3가 <Response [200]>
1161 충무로 파주옥 <Response [200]>
1162 분카샤 <Response [200]>
1163 퍼퓰러 <Response [200]>
1164 을지다락 <Response [200]>
1165 을지로미팅룸 <Response [200]>
1166 을지로 차이 <Response [200]>
1167 산수갑산 <Response [200]>
1168 안동장 <Response [200]>
1169 무월식탁 을지로 <Response [200]>
1170 임마지노 <Response [200]>
1171 동원집 <Response [200]>
1172 죠지 <Response [200]>
1173 우래옥 <Response [200]>
1174 십분의일 <Response [200]>
1175 커피한약방 <Response [200]>
1176 다케오 호르몬 데판야끼 <Response [200]>
1177 호랑이 <Response [200]>
1178 을지다방 <Response [200]>
1179 줄리아 <Response [200]>
1180 을지오뎅 <Response [200]>
1181 양미옥 을지로본점 <Response [200]>
1182 진작 <Response [200]>
1183 을지로쎄시봉 <Response [200]>
1184 장수보쌈 <Response [200]>
1185 조선옥 <Response [200]>
1186 만선호프 노가리체인본점 <Response [200]>
1187 청와옥 을지로3가직영점 <Response [200]>
1188 after jerk off <Response [200]>
1189 동경우동 <Response [200]>
1190 오카구라 라멘

...

20279 팀호완 잠실점 <Response [200]>
20280 큰손닭한마리 <Response [200]>
20281 명서식당 <Response [200]>
20282 이치고 <Response [200]>
20283 매드포갈릭 롯데시네마월드타워점 <Response [200]>
20284 브루클린더버거조인트 롯데월드몰점 <Response [200]>
20285 크래프트아일랜드 잠실점 <Response [200]>
20286 애슐리퀸즈 잠실롯데캐슬점 <Response [200]>
20287 요리하는남자 <Response [200]>
20288 도쿄등심 롯데캐슬잠실점 <Response [200]>
20289 나무수 <Response [200]>
20290 파오파오 <Response [200]>
20291 세이류 <Response [200]>
20292 정순함박 <Response [200]>
20293 소마드로잉카페 잠실새내점 <Response [200]>
20294 갓덴스시 롯데월드몰점 <Response [200]>
20295 한점 <Response [200]>
20296 시그니엘서울 바81 <Response [200]>
20297 교양식사 <Response [200]>
20298 신천포차 <Response [200]>
20299 브럭시 <Response [200]>
20300 나무수 <Response [200]>
20301 광연수산 <Response [200]>
20302 명태어장 강일점 <Response [200]>
20303 화이트글래스 <Response [200]>
20304 버드나무집 <Response [200]>
20305 차오차오 <Response [200]>
20306 신의주찹쌀순대 강일동점 <Response [200]>
20307 장어집 <Response [200]>
20308 느티나무집 <Response [200]>
20309 처갓집양념치킨 강일점 <Response [200]>
20310 쌀통닭 강일점 <Response [200]>
20311 쭈

20533 BBQ 고덕점 <Response [200]>
20534 원두막 <Response [200]>
20535 도미노피자 강일점 <Response [200]>
20536 스탁 <Response [200]>
20537 33떡볶이 고덕그라시움점 <Response [200]>
20538 이디야커피 고덕그라시움후문점 <Response [200]>
20539 신떡순 신천할매떡볶이 고덕점 <Response [200]>
20540 가츠몽 상일역점 <Response [200]>
20541 쿠웨이트 떡볶이 <Response [200]>
20542 린칭 <Response [200]>
20543 후라이드참잘하는집 <Response [200]>
20544 만나냉면 <Response [200]>
20545 33떡볶이 고덕래미안힐스점 <Response [200]>
20546 그라시움 옛날통닭 <Response [200]>
20547 59년왕십리 <Response [200]>
20548 도담도담식빵 고덕점 <Response [200]>
20549 굽네치킨 고덕1,2동점 <Response [200]>
20550 스테이지28 <Response [200]>
20551 스시정원 <Response [200]>
20552 오늘도 만족 <Response [200]>
20553 펠어커피 고덕점 <Response [200]>
20554 백두감자탕 <Response [200]>
20555 개성김치손만두 <Response [200]>
20556 카페제이 <Response [200]>
20557 곱분이곱창 상일동역점 <Response [200]>
20558 버거타임 <Response [200]>
20559 리틀방콕 고덕점 <Response [200]>
20560 33Acre <Response [200]>
20561 블랙워터로스터스 <Response [200]>
20562 여수집 <Response [200]>
20563 와인살롱 고덕살롱 <Response [200]>
20564 홍담족발 고덕점 <Respo

20804 성내볼데기집 <Response [200]>
20805 생어거스틴 성내점 <Response [500]>
20805 생어거스틴 성내점 <Response [500]>
20806 버섯잔치집 <Response [500]>
20806 버섯잔치집 <Response [500]>
20807 블랑제리11-17 <Response [200]>
20808 풍년상회 <Response [200]>
20809 브랜뉴하이몬드 <Response [200]>
20810 긴자 올림픽점 <Response [200]>
20811 YUP <Response [200]>
20812 커피몽타주 <Response [200]>
20813 미드데이썬 <Response [200]>
20814 WAAAH <Response [200]>
20815 갓잇 올림픽공원점 <Response [200]>
20816 수라연 <Response [200]>
20817 먼데이블루스 성내점 <Response [200]>
20818 피콜로 <Response [200]>
20819 안녕식당 2호점 <Response [200]>
20820 펠어커피 강동점 <Response [200]>
20821 반상회 <Response [200]>
20822 우양우 <Response [200]>
20823 에이컷스테이크 올림픽공원점 <Response [200]>
20824 철판살롱 <Response [200]>
20825 화려한곱창 <Response [200]>
20826 이십다시오 <Response [200]>
20827 셀프하우스 <Response [200]>
20828 키친사계 <Response [200]>
20829 길위의 파스타 둔촌점 <Response [200]>
20830 채스우드커피 <Response [200]>
20831 램니쿠야 둔촌점 <Response [200]>
20832 통로 <Response [500]>
20832 통로 <Response [500]>
20833 온도계 <Response [200]>
20834 시집그릴하우스

21062 오븐베이크 <Response [200]>
21063 유로스타커피 <Response [200]>
21064 조게찜터 <Response [200]>
21065 드딜라이트 <Response [200]>
21066 암사고바우감자탕 <Response [200]>
21067 지리산왕재봉흑돼지 <Response [200]>
21068 풍촌코다리네 본점 <Response [200]>
21069 카페선사 <Response [200]>
21070 렛미베이크 <Response [200]>
21070 렛미베이크 <Response [200]>
21071 공간 <Response [200]>
21071 공간 <Response [200]>
21072 제주항 <Response [200]>
21073 팔오수산 <Response [200]>
21074 빙삼 <Response [200]>
21075 백채 암사역점 <Response [200]>
21076 이토야 본점 <Response [200]>
21077 황실짜장 <Response [200]>
21078 우리동네막국수국밥 <Response [200]>
21079 빨간쭈꾸미 본점 <Response [200]>
21080 봉쥬르 쟝딸롱 <Response [200]>
21081 랍스타킹 <Response [200]>
21082 버거팜 <Response [200]>
21083 초롱이고모부대찌개 <Response [200]>
21084 해안반점 <Response [200]>
21085 카페공공장소 <Response [200]>
21086 황가네곱창 <Response [200]>
21087 선남sea <Response [200]>
21088 와줘서고마워요 <Response [200]>
21089 마이플레이버 <Response [200]>
21090 본새커피 <Response [200]>
21091 스타벅스 강동암사DT점 <Response [200]>
21092 The라김 라면 김밥 <Response [200]>
21093 스시연 <Respons

KeyError: 21200

In [5]:
df.head()

,id,dong,name,tel,addr,bizhour,category,context,menu,reviewcount,microreview,thumbnail,lng,lat,rating
0,37450428,사직동,박순례 손말이고기 산정집 광화문점,02-310-9636,서울특별시 종로구 당주동 3-2 선덕빌딩 2층,일요일 휴무 명절연휴 휴무 | 매일 11:30~14:00 점심 | 매일 14:00~...,"['한식', '소고기구이']","['시래기', '고기말이', '한우', '고급스러운', '나들이']","한우말이고기 21,000 | 한우내장볶음 21,000 | 한우내장전골 25,000 ...",203,[],https://ldb-phinf.pstatic.net/20200930_73/1601...,126.974200,37.572380,4.41
1,1934318307,사직동,한옥 달,010-8706-3392,서울특별시 종로구 체부동 68,평일 11:30~22:00 (L.O. 20:30) | 평일 15:30~17:00 B...,"['이탈리아음식', '스파게티,파스타전문']",[],"단호박 크림 스프 15,000 | 아보카도 허머스 16,000 | 리코타 치즈 샐러...",322,[],https://ldb-phinf.pstatic.net/20190808_53/1565...,126.971983,37.576946,4.32
2,19489992,사직동,할매집,02-735-2608,서울특별시 종로구 내자동 108,매일 12:00~21:30,"['한식', '족발,보쌈']","['감자탕', '족발', '미슐랭가이드', '우아한', '콩국수']","족발 30,000 | 감자탕(중) 25,000 | 감자탕(대) 30,000",390,"['콩나물 넣은 감자탕', '구수하고 푸짐한 감자탕을 먹을 수 있는 식신로드 맛집'...",http://blogfiles.naver.net/20150910_18/xxxyeji...,126.971475,37.575660,4.3
3,1010940645,사직동,디퍼카페테리아,010-9160-4482,서울특별시 종로구 체부동 2-2,주말 12:00~19:30 | 수요일 12:00~19:30 | 목요일 12:00~1...,"['음식점', '카페,디저트']",[],"아메리카노 3,500 | 카페라떼 4,000 | 그라니따 5,500 | 스폴리아텔라...",39,[],https://ldb-phinf.pstatic.net/20200908_251/159...,126.970393,37.579038,4.48
4,676217924,사직동,후라토식당 경복궁 본점,02-730-7090,서울특별시 종로구 당주동 5,매일 11:30~22:00 평일 브레이크타임 15:00-17:00,"['일식', '일식당']","['오므라이스', '데이트', '일본가정식', '프라이빗', '돈가스']","규카츠 15,000 | 반숙 오므라이스 11,000 | 야마가타 민치카레 12,00...",497,[],https://ldb-phinf.pstatic.net/20200804_260/159...,126.974311,37.572032,4.67
